# Analytical Feature Engineering via Mathematical Programming

## Overview
This framework addresses feature engineering for crude oil futures forecasting through rigorous mathematical optimization. We propose a three-stage pipeline:

$$\text{Feature Selection (MIP)} \rightarrow \text{Parameter Tuning (SPSA)} \rightarrow \text{Signal Synthesis (QP)}$$

Unlike conventional heuristic approaches (Stepwise Regression, LASSO with fixed penalties), each stage solves a well-defined optimization problem guaranteeing locally/globally optimal solutions under stated constraints.

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import gurobipy as gp
from gurobipy import GRB
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

In [3]:

def calc_rsi(series, period=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).ewm(alpha=1/period, adjust=False).mean()
    loss = (-delta.where(delta < 0, 0)).ewm(alpha=1/period, adjust=False).mean()
    return 100 - (100 / (1 + gain / loss))

def calc_macd(series, fast=12, slow=26, signal=9):
    exp1 = series.ewm(span=fast, adjust=False).mean()
    exp2 = series.ewm(span=slow, adjust=False).mean()
    macd = exp1 - exp2
    sig = macd.ewm(span=signal, adjust=False).mean()
    return pd.concat([macd.rename('MACD'), sig.rename('MACD_Signal')], axis=1)

def calc_bbands(series, length=20, std=2):
    sma = series.rolling(window=length).mean()
    rstd = series.rolling(window=length).std()
    return pd.concat([(sma + std*rstd).rename('BBU'), sma.rename('BBM'), (sma - std*rstd).rename('BBL')], axis=1)

def calc_atr(df, length=14):
    h, l, c = df['High'], df['Low'], df['Close']
    tr = pd.concat([h - l, (h - c.shift()).abs(), (l - c.shift()).abs()], axis=1).max(axis=1)
    return tr.ewm(alpha=1/length, adjust=False).mean()


## Data Preprocessing & Problem Setup

Let $\mathcal{D} = \{(p_t, v_t, h_t, l_t, o_t)\}_{t=1}^{T}$ denote the raw OHLCV data for crude oil futures over $T$ trading days.

**Definition 1 (Feature Matrix):** Let $X \in \mathbb{R}^{n \times p}$ denote the feature matrix constructed from technical indicators, where $n = T - d_{\max}$ is the number of available samples after accounting for indicator warm-up period, and $p$ is the number of features.

**Definition 2 (Target Variable):** The regression target is the forward-looking log return:
$$y_t = \log\left(\frac{p_{t+1}}{p_t}\right), \quad y \in \mathbb{R}^n$$

**Standardization:** To ensure numerical stability in convex optimization, we apply Z-score normalization:
$$X_\text{std} = \frac{X - \mathbf{1}\mu^\top}{\sigma}, \quad \text{where} \ \mu \in \mathbb{R}^p, \ \sigma \in \mathbb{R}^p$$
Likewise, $y_\text{std} = \frac{y - \bar{y}}{\text{Std}(y)}$.

In [4]:

print(">>> Loading Crude Oil Data...")
df = yf.download("CL=F", start="2020-01-01", end="2024-01-01", progress=False)
if isinstance(df.columns, pd.MultiIndex): df.columns = df.columns.get_level_values(0)

if df.empty: raise ValueError("Data download failed.")

# Feature Engineering
df['RSI'] = calc_rsi(df['Close'])
df = pd.concat([df, calc_macd(df['Close']), calc_bbands(df['Close'])], axis=1)
df['ATR'] = calc_atr(df)
df['Ret_1d'] = df['Close'].pct_change()
df['Target'] = df['Close'].pct_change().shift(-1) 

df.dropna(inplace=True)

# Matrix setup
feat_cols = [c for c in df.columns if c not in ['Open','High','Low','Close','Adj Close','Volume','Target']]
X_raw = df[feat_cols].values
y_raw = df['Target'].values

scaler = StandardScaler()
X = scaler.fit_transform(X_raw)
y = (y_raw - np.mean(y_raw)) / np.std(y_raw)
n, p = X.shape
print(f"Data Ready: {n} samples, {p} features.")

# Pre-compute Matrices for MIP (Speed up)
Q_all = X.T @ X
c_all = -2 * y.T @ X


>>> Loading Crude Oil Data...


/var/folders/_x/1pnr6wwx33nd4yqyttygrv580000gn/T/ipykernel_63751/1624841446.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download("CL=F", start="2020-01-01", end="2024-01-01", progress=False)


Data Ready: 986 samples, 8 features.


## Stage 1: Global Optimal Feature Selection (MIQP)

**Problem 1.1 (Baseline Fixed Cardinality):**

We solve the Mixed-Integer Quadratic Program:

$$\min_{\beta, z} \|y - X\beta\|_2^2 \quad \text{subject to} \quad \begin{cases}
\sum_{i=1}^p z_i = k^* \\
|\beta_i| \le M z_i, \quad i = 1, \ldots, p \\
z_i \in \{0, 1\}, \quad \beta_i \in \mathbb{R}
\end{cases}$$

where $M > 0$ is a sufficiently large constant (Big-M). The binary indicator $z_i = 1$ if feature $i$ is selected; $z_i = 0$ forces $\beta_i = 0$. This formulation guarantees **global optimality** by exhaustively searching the feature selection space under the cardinality constraint, unlike greedy heuristics (Stepwise Regression).

In [5]:

print("\n>>> [Baseline] Running Basic MIP with fixed k=5...")

m_base = gp.Model("MIP_Base")
m_base.setParam('OutputFlag', 0)
beta = m_base.addMVar(p, lb=-GRB.INFINITY, name="beta")
z = m_base.addMVar(p, vtype=GRB.BINARY, name="z")

m_base.setObjective(beta @ Q_all @ beta + c_all @ beta, GRB.MINIMIZE)

# Hard Constraint: Exactly 5 features
m_base.addConstr(z.sum() == 5)
for i in range(p): # Big-M
    m_base.addConstr(beta[i] <= 10 * z[i])
    m_base.addConstr(beta[i] >= -10 * z[i])

m_base.optimize()

indices_base = [i for i in range(p) if z[i].X > 0.5]
print(f"   Baseline Selected (k=5): {[feat_cols[i] for i in indices_base]}")




>>> [Baseline] Running Basic MIP with fixed k=5...
Restricted license - for non-production use only - expires 2026-11-23
   Baseline Selected (k=5): ['MACD', 'MACD_Signal', 'BBM', 'BBL', 'Ret_1d']
   Baseline Selected (k=5): ['MACD', 'MACD_Signal', 'BBM', 'BBL', 'Ret_1d']


## Stage 1b: Automated Cardinality Selection via Time-Series CV

**Problem 1.2 (Data-Driven Cardinality):**

To avoid ad-hoc specification of $k$, we perform temporal cross-validation:

Given a sequence of folds $\{(I_\text{train}^j, I_\text{val}^j)\}_{j=1}^{J}$ satisfying $I_\text{train}^j \prec I_\text{val}^j$ (temporal ordering), for each candidate $k \in \{1, 2, \ldots, 8\}$:

$$\begin{align}
\text{ValMSE}(k) &= \frac{1}{J}\sum_{j=1}^{J} \left\| y_{I_{\text{val}}^j} - X_{I_{\text{val}}^j} \hat{\beta}^j(k) \right\|_2^2\\[6pt]
k^* &= \arg\min_{k} \text{ValMSE}(k)
\end{align}$$

where $\hat{\beta}^j(k)$ solves Problem 1.1 on the $j$-th training fold. This procedure performs **empirical model selection** respecting temporal causality, mitigating both overfitting and the arbitrary choice of cardinality.

In [6]:

print("\n>>> [Advanced A] Running Time-Series CV to find optimal k (1..8)...")

tscv = TimeSeriesSplit(n_splits=3)
results_k = {}

for k in range(1, 9):
    val_errors = []
    for train_ix, val_ix in tscv.split(X):
        X_tr, X_val = X[train_ix], X[val_ix]
        y_tr, y_val = y[train_ix], y[val_ix]
        
        m = gp.Model("MIP_CV")
        m.setParam('OutputFlag', 0)
        b = m.addMVar(p, lb=-GRB.INFINITY)
        zz = m.addMVar(p, vtype=GRB.BINARY)
        
        # Train Objective
        Q_tr = X_tr.T @ X_tr
        c_tr = -2 * y_tr.T @ X_tr
        m.setObjective(b @ Q_tr @ b + c_tr @ b, GRB.MINIMIZE)
        
        m.addConstr(zz.sum() == k)
        for i in range(p):
            m.addConstr(b[i] <= 10 * zz[i])
            m.addConstr(b[i] >= -10 * zz[i])
            
        m.optimize()
        
        if m.Status == GRB.OPTIMAL:
            mse = mean_squared_error(y_val, X_val @ b.X)
            val_errors.append(mse)
            
    avg_mse = np.mean(val_errors)
    results_k[k] = avg_mse
    print(f"   k={k}, Val MSE: {avg_mse:.4f}")

best_k = min(results_k, key=results_k.get)
print(f"   >>> Optimal k determined by CV: {best_k}")

# Retrain Final Model with Optimal k
m_final = gp.Model("MIP_Final")
m_final.setParam('OutputFlag', 0)
beta = m_final.addMVar(p, lb=-GRB.INFINITY)
z = m_final.addMVar(p, vtype=GRB.BINARY)
m_final.setObjective(beta @ Q_all @ beta + c_all @ beta, GRB.MINIMIZE)
m_final.addConstr(z.sum() == best_k)
for i in range(p):
    m_final.addConstr(beta[i] <= 10 * z[i])
    m_final.addConstr(beta[i] >= -10 * z[i])
m_final.optimize()

indices_cv = [i for i in range(p) if z[i].X > 0.5]
print(f"   Advanced A Selected (k={best_k}): {[feat_cols[i] for i in indices_cv]}")




>>> [Advanced A] Running Time-Series CV to find optimal k (1..8)...
   k=1, Val MSE: 0.0546
   k=2, Val MSE: 0.0670
   k=3, Val MSE: 0.0882
   k=4, Val MSE: 0.1093
   k=5, Val MSE: 0.1269
   k=6, Val MSE: 0.1254
   k=7, Val MSE: 0.1305
   k=8, Val MSE: 0.1305
   >>> Optimal k determined by CV: 1
   Advanced A Selected (k=1): ['Ret_1d']


## Stage 1c: Soft Cardinality via $L_0$-Regularization

**Problem 1.3 (Penalized Objective):**

As an alternative to hard constraints, we employ $L_0$-norm regularization:

$$\begin{align}
\min_{\beta \in \mathbb{R}^p, z \in \{0,1\}^p} &\quad \|y - X\beta\|_2^2 + \lambda \cdot n \sum_{i=1}^p z_i \tag{Lagrangian}\\[6pt]
\text{s.t.} &\quad |\beta_i| \le M z_i, \quad i = 1, \ldots, p
\end{align}$$

where $\lambda > 0$ is a regularization parameter representing the marginal cost of feature inclusion. The solver autonomously balances prediction error against model complexity. This is analogous to information criteria (AIC/BIC): minimizing $\text{RSS} + 2p\hat{\sigma}^2$ versus $\text{RSS} + \lambda np$.

In [7]:

print("\n>>> [Advanced B] Running Penalized MIP (Lagrangian)...")

lambda_pen = 0.05 # Cost per feature
m_pen = gp.Model("MIP_Pen")
m_pen.setParam('OutputFlag', 0)
beta_p = m_pen.addMVar(p, lb=-GRB.INFINITY)
z_p = m_pen.addMVar(p, vtype=GRB.BINARY)

# Objective: MSE + lambda * num_features
obj = (beta_p @ Q_all @ beta_p + c_all @ beta_p) + (lambda_pen * n * z_p.sum())
m_pen.setObjective(obj, GRB.MINIMIZE)

for i in range(p):
    m_pen.addConstr(beta_p[i] <= 10 * z_p[i])
    m_pen.addConstr(beta_p[i] >= -10 * z_p[i])

m_pen.optimize()

indices_pen = [i for i in range(p) if z_p[i].X > 0.5]
print(f"   Advanced B Selected (lambda={lambda_pen}): {[feat_cols[i] for i in indices_pen]}")

# Use CV results for downstream tasks
final_best_features = [feat_cols[i] for i in indices_cv]



>>> [Advanced B] Running Penalized MIP (Lagrangian)...
   Advanced B Selected (lambda=0.05): ['Ret_1d']
   Advanced B Selected (lambda=0.05): ['Ret_1d']


## Stage 2: Gradient-Free Hyperparameter Tuning (SPSA)

**Problem 2 (Black-Box Parameter Optimization):**

Technical indicators typically depend on integer hyperparameters (e.g., RSI window $w$) where the objective $\rho(w) = |\text{Corr}(\\text{RSI}_w, y)|$ is non-smooth and non-differentiable.

We employ **Simultaneous Perturbation Stochastic Approximation (SPSA)**:

$$\begin{align}
\theta_{k+1} &= \theta_k - a_k \hat{g}_k(\theta_k)\\[6pt]
\hat{g}_k(\theta_k) &= \frac{L(\theta_k + c_k \Delta_k) - L(\theta_k - c_k \Delta_k)}{2 c_k \Delta_k}
\end{align}$$

where $\Delta_k \sim \text{Bernoulli}(\{-1, +1\})$, $a_k = a/(k+1)^{0.602}$, $c_k = c/(k+1)^{0.101}$ are decreasing step sizes. 

**Advantage:** SPSA requires only **2 function evaluations per iteration** versus $2p$ for Finite Differences, enabling efficient tuning of RSI window to maximize $-|\\text{Corr}(\\text{RSI}_{\\theta}, y)|$."

In [8]:

print("\n>>> Running SPSA for RSI Parameter...")
theta = np.array([14.0])
for k in range(30):
    ck = 1.0 / (k + 1)**0.101
    ak = 2.0 / (k + 1 + 10)**0.602
    delta = np.random.choice([-1, 1])
    
    w_p, w_m = max(2, int(theta[0]+ck*delta)), max(2, int(theta[0]-ck*delta))
    
    # Loss: -Correlation
    l_p = -abs(np.corrcoef(calc_rsi(df['Close'], w_p).fillna(0), df['Target'].fillna(0))[0,1])
    l_m = -abs(np.corrcoef(calc_rsi(df['Close'], w_m).fillna(0), df['Target'].fillna(0))[0,1])
    
    theta = theta - ak * (l_p - l_m)/(2*ck*delta)

best_win = max(2, int(theta[0]))
print(f"   SPSA Optimized RSI Window: {best_win}")
df[f'RSI_SPSA_{best_win}'] = calc_rsi(df['Close'], best_win)



>>> Running SPSA for RSI Parameter...
   SPSA Optimized RSI Window: 14
   SPSA Optimized RSI Window: 14


## Stage 3: Optimal Signal Synthesis (Convex QP)

**Problem 3 (Mean-Variance Super-Factor):**

Inspired by Modern Portfolio Theory, we synthesize a composite predictor by solving:

$$\min_{w \in \mathbb{R}^m} w^\top \Sigma w \quad \text{subject to} \quad \begin{cases}
\sum_{i=1}^m w_i = 1 \\
w^\top \rho \ge \tau \\
w_i \ge 0, \quad i = 1, \ldots, m
\end{cases}$$

where $\Sigma = \text{Cov}(X_{\text{selected}})$ is the covariance matrix of selected features, $\rho \in \mathbb{R}^m$ is the feature-target correlation vector, $\tau = 0.5 \cdot \max_i |\rho_i|$ is a lower bound on predictive correlation, and $w$ represents normalized portfolio weights. This convex optimization problem maximizes Signal-to-Noise Ratio (SNR): $\text{SNR} = \frac{\rho^\top w}{\sqrt{w^\top \Sigma w}}$.

In [9]:

print("\n>>> Running QP Super-Factor...")
cols_qp = final_best_features + [f'RSI_SPSA_{best_win}']
X_qp = scaler.fit_transform(df[cols_qp].fillna(0).values)
n_qp, p_qp = X_qp.shape

Sigma = np.cov(X_qp, rowvar=False)
xy_cov = np.array([np.cov(X_qp[:,i], y)[0,1] for i in range(p_qp)])

m_qp = gp.Model("QP")
m_qp.setParam('OutputFlag', 0)
w = m_qp.addMVar(p_qp, lb=0.0)

m_qp.setObjective(w @ Sigma @ w, GRB.MINIMIZE)
m_qp.addConstr(w.sum() == 1)
m_qp.addConstr(w @ xy_cov >= np.max(np.abs(xy_cov)) * 0.5)

m_qp.optimize()

if m_qp.Status == GRB.OPTIMAL:
    df['Super_Factor_QP'] = X_qp @ w.X
    print("   QP Solved. Super-Factor Created.")
else:
    print("   QP Failed.")



>>> Running QP Super-Factor...
   QP Solved. Super-Factor Created.
   QP Solved. Super-Factor Created.


In [10]:

out_cols = ['Close', 'Target'] + [f'RSI_SPSA_{best_win}', 'Super_Factor_QP']
print("\n>>> Final Data Tail:")
print(df[out_cols].dropna().tail())


>>> Final Data Tail:
                Close    Target  RSI_SPSA_14  Super_Factor_QP
Date                                                         
2023-12-21  73.889999 -0.004466    49.138127        -0.136771
2023-12-22  73.559998  0.027325    48.144088        -0.176060
2023-12-26  75.570000 -0.019320    54.218937         0.205678
2023-12-27  74.110001 -0.031575    49.667483        -0.182414
2023-12-28  71.769997 -0.001672    43.381764        -0.485098


In [11]:

# === Comprehensive Summary of Feature Selection Results ===

print("\n" + "="*80)
print("FEATURE SELECTION SUMMARY")
print("="*80)

# Prepare correlation data
X_all = scaler.fit_transform(df[feat_cols].values)
correlations_all = np.array([np.corrcoef(X_all[:,i], y)[0,1] for i in range(p)])

# 1. Baseline MIP Results
print("\n[Method 1] Baseline MIP (Fixed k=5)")
print("-" * 60)
baseline_features = [feat_cols[i] for i in indices_base]
print(f"Selected Features: {baseline_features}")
print(f"Number of Features: {len(baseline_features)}")
baseline_corrs = [correlations_all[feat_cols.index(f)] for f in baseline_features]
print(f"Individual Correlations: {[f'{c:.4f}' for c in baseline_corrs]}")
print(f"Avg |Correlation|: {np.mean(np.abs(baseline_corrs)):.4f}")
print(f"Max |Correlation|: {np.max(np.abs(baseline_corrs)):.4f}")

# 2. Time-Series CV Results
print("\n[Method 2] Time-Series CV (Data-Driven k)")
print("-" * 60)
cv_features = [feat_cols[i] for i in indices_cv]
print(f"Optimal Cardinality: k* = {best_k}")
print(f"Selected Features: {cv_features}")
print(f"Number of Features: {len(cv_features)}")
cv_corrs = [correlations_all[feat_cols.index(f)] for f in cv_features]
print(f"Individual Correlations: {[f'{c:.4f}' for c in cv_corrs]}")
print(f"Avg |Correlation|: {np.mean(np.abs(cv_corrs)):.4f}")
print(f"Max |Correlation|: {np.max(np.abs(cv_corrs)):.4f}")
print(f"Validation MSE: {results_k[best_k]:.6f}")

# 3. Penalized MIP Results
print("\n[Method 3] Penalized MIP (L0-Regularization, λ={})".format(lambda_pen))
print("-" * 60)
pen_features = [feat_cols[i] for i in indices_pen]
print(f"Selected Features: {pen_features}")
print(f"Number of Features: {len(pen_features)}")
pen_corrs = [correlations_all[feat_cols.index(f)] for f in pen_features]
print(f"Individual Correlations: {[f'{c:.4f}' for c in pen_corrs]}")
print(f"Avg |Correlation|: {np.mean(np.abs(pen_corrs)):.4f}")
print(f"Max |Correlation|: {np.max(np.abs(pen_corrs)):.4f}")

# Feature Overlap Analysis
print("\n" + "="*80)
print("OVERLAP ANALYSIS")
print("="*80)
baseline_set = set(baseline_features)
cv_set = set(cv_features)
pen_set = set(pen_features)

overlap_base_cv = baseline_set & cv_set
overlap_base_pen = baseline_set & pen_set
overlap_cv_pen = cv_set & pen_set
overlap_all = baseline_set & cv_set & pen_set

print(f"\nBaseline ∩ CV:       {overlap_base_cv} ({len(overlap_base_cv)} features)")
print(f"Baseline ∩ Penalized: {overlap_base_pen} ({len(overlap_base_pen)} features)")
print(f"CV ∩ Penalized:       {overlap_cv_pen} ({len(overlap_cv_pen)} features)")
print(f"All Three Methods:    {overlap_all} ({len(overlap_all)} features)")

# QP Super-Factor Performance
print("\n" + "="*80)
print("SUPER-FACTOR SYNTHESIS (QP)")
print("="*80)
print(f"Input Features: {cols_qp}")
print(f"Number of Input Features: {len(cols_qp)}")
super_factor_corr = np.corrcoef(df['Super_Factor_QP'].fillna(0), y)[0,1]
print(f"Super-Factor Correlation with Target: {super_factor_corr:.4f}")
print(f"Super-Factor Variance: {np.var(df['Super_Factor_QP'].fillna(0)):.6f}")
print(f"\nQP Weights (Importance):")
for fname, wt in zip(cols_qp, w.X):
    print(f"  {fname:20s}: {wt:.4f}")

# Overall Comparison
print("\n" + "="*80)
print("PERFORMANCE RANKING")
print("="*80)
methods_perf = [
    ("Baseline MIP (k=5)", np.mean(np.abs(baseline_corrs)), len(baseline_features)),
    ("Time-Series CV (k={})".format(best_k), np.mean(np.abs(cv_corrs)), len(cv_features)),
    ("Penalized MIP (λ={})".format(lambda_pen), np.mean(np.abs(pen_corrs)), len(pen_features)),
    ("Super-Factor (QP)", abs(super_factor_corr), 1)
]

ranked = sorted(methods_perf, key=lambda x: x[1], reverse=True)
for rank, (name, corr, n_feat) in enumerate(ranked, 1):
    print(f"{rank}. {name:40s} | Avg |ρ|: {corr:.4f} | #Features: {n_feat}")

print("\n" + "="*80)



FEATURE SELECTION SUMMARY

[Method 1] Baseline MIP (Fixed k=5)
------------------------------------------------------------
Selected Features: ['MACD', 'MACD_Signal', 'BBM', 'BBL', 'Ret_1d']
Number of Features: 5
Individual Correlations: ['0.0785', '0.0733', '0.0527', '0.0502', '0.3041']
Avg |Correlation|: 0.1118
Max |Correlation|: 0.3041

[Method 2] Time-Series CV (Data-Driven k)
------------------------------------------------------------
Optimal Cardinality: k* = 1
Selected Features: ['Ret_1d']
Number of Features: 1
Individual Correlations: ['0.3041']
Avg |Correlation|: 0.3041
Max |Correlation|: 0.3041
Validation MSE: 0.054627

[Method 3] Penalized MIP (L0-Regularization, λ=0.05)
------------------------------------------------------------
Selected Features: ['Ret_1d']
Number of Features: 1
Individual Correlations: ['0.3041']
Avg |Correlation|: 0.3041
Max |Correlation|: 0.3041

OVERLAP ANALYSIS

Baseline ∩ CV:       {'Ret_1d'} (1 features)
Baseline ∩ Penalized: {'Ret_1d'} (1 featur

In [ ]:

# === Best Method Selection & Recommendation ===

print("\n" + "="*80)
print("BEST METHOD SELECTION")
print("="*80)

# Comprehensive scoring system
scores = {}

# 1. Baseline MIP Score
baseline_score = {
    'method': 'Baseline MIP (k=5)',
    'avg_corr': np.mean(np.abs(baseline_corrs)),
    'max_corr': np.max(np.abs(baseline_corrs)),
    'num_features': len(baseline_features),
    'interpretability': 9,  # Very interpretable (fixed k=5)
    'stability': 6,  # May not be optimal for actual data
    'is_data_driven': False
}

# 2. Time-Series CV Score
cv_score = {
    'method': f'Time-Series CV (k*={best_k})',
    'avg_corr': np.mean(np.abs(cv_corrs)),
    'max_corr': np.max(np.abs(cv_corrs)),
    'num_features': len(cv_features),
    'interpretability': 8,  # Good interpretability with data-driven k
    'stability': 9,  # Respects temporal causality, OOS validation
    'is_data_driven': True,
    'validation_mse': results_k[best_k]
}

# 3. Penalized MIP Score
pen_score = {
    'method': f'Penalized MIP (λ={lambda_pen})',
    'avg_corr': np.mean(np.abs(pen_corrs)),
    'max_corr': np.max(np.abs(pen_corrs)),
    'num_features': len(pen_features),
    'interpretability': 7,  # Less transparent (regularization parameter tuning)
    'stability': 7,  # Auto-balances complexity but λ is fixed
    'is_data_driven': False
}

# 4. Super-Factor (QP) Score
qp_score = {
    'method': 'Super-Factor (QP)',
    'avg_corr': abs(super_factor_corr),
    'max_corr': abs(super_factor_corr),
    'num_features': len(cols_qp),
    'interpretability': 5,  # Black-box linear combination
    'stability': 8,  # Convex, globally optimal
    'is_data_driven': True,
    'is_ensemble': True
}

methods = [baseline_score, cv_score, pen_score, qp_score]

# Calculate composite scores (weighted)
print("\nDetailed Scores:")
print("-" * 80)
for method in methods:
    corr_weight = method['avg_corr']
    stability_weight = method['stability'] / 10.0
    interp_weight = method['interpretability'] / 10.0
    
    # Composite score: 40% predictive power + 30% stability + 30% interpretability
    composite = 0.4 * corr_weight + 0.3 * stability_weight + 0.3 * interp_weight
    method['composite_score'] = composite
    
    print(f"\n{method['method']}")
    print(f"  Avg |Correlation|:   {method['avg_corr']:.4f}")
    print(f"  Max |Correlation|:   {method['max_corr']:.4f}")
    print(f"  # Features:          {method['num_features']}")
    print(f"  Interpretability:    {method['interpretability']}/10")
    print(f"  Stability:           {method['stability']}/10")
    print(f"  Composite Score:     {composite:.4f}")

# Rank by different criteria
print("\n" + "="*80)
print("RANKINGS BY CRITERION")
print("="*80)

# Ranking 1: Pure Predictive Power
print("\n1. Predictive Power (Avg |Correlation|):")
ranked_corr = sorted(methods, key=lambda x: x['avg_corr'], reverse=True)
for i, m in enumerate(ranked_corr, 1):
    print(f"   {i}. {m['method']:45s} → {m['avg_corr']:.4f}")

# Ranking 2: Stability & Robustness
print("\n2. Stability & Robustness:")
ranked_stab = sorted(methods, key=lambda x: x['stability'], reverse=True)
for i, m in enumerate(ranked_stab, 1):
    print(f"   {i}. {m['method']:45s} → {m['stability']}/10")

# Ranking 3: Interpretability
print("\n3. Interpretability:")
ranked_interp = sorted(methods, key=lambda x: x['interpretability'], reverse=True)
for i, m in enumerate(ranked_interp, 1):
    print(f"   {i}. {m['method']:45s} → {m['interpretability']}/10")

# Ranking 4: Composite Score
print("\n4. Composite Score (Balanced):")
ranked_composite = sorted(methods, key=lambda x: x['composite_score'], reverse=True)
for i, m in enumerate(ranked_composite, 1):
    print(f"   {i}. {m['method']:45s} → {m['composite_score']:.4f}")

# RECOMMENDATION
best_method = ranked_composite[0]
second_best = ranked_composite[1]

print("\n" + "="*80)
print("FINAL RECOMMENDATION")
print("="*80)

print(f"\n BEST METHOD: {best_method['method']}")
print(f"   Composite Score: {best_method['composite_score']:.4f}")
print(f"   Predictive Power: {best_method['avg_corr']:.4f}")
print(f"   Stability: {best_method['stability']}/10")

if best_method['is_data_driven']:
    print(f"   ✓ Data-Driven: YES (learns optimal cardinality from data)")
    if 'validation_mse' in best_method:
        print(f"   ✓ Validation MSE: {best_method['validation_mse']:.6f}")
else:
    print(f"   • Data-Driven: NO (uses fixed hyperparameters)")

print(f"\n Runner-up: {second_best['method']}")
print(f"   Composite Score: {second_best['composite_score']:.4f}")

# Use case recommendations
print("\n" + "="*80)
print("USE CASE RECOMMENDATIONS")
print("="*80)



# Export recommendation
print("\n" + "="*80)
print("RECOMMENDED FACTORS FOR DEPLOYMENT")
print("="*80)

if best_method['method'].startswith('Time-Series CV'):
    print(f"\n✓ Selected Method: Time-Series CV with k* = {best_k}")
    print(f"✓ Recommended Factors: {cv_features}")
    print(f"✓ Number of Factors: {len(cv_features)}")
    print(f"✓ Individual Correlations:")
    for feat, corr in zip(cv_features, cv_corrs):
        print(f"    {feat:20s}: {corr:+.4f}")
    recommended_factors = cv_features
    recommended_corrs = cv_corrs
elif best_method['is_ensemble']:
    print(f"\n✓ Selected Method: Super-Factor (QP)")
    print(f"✓ Input Factors: {cols_qp}")
    print(f"✓ Output: Single composite factor 'Super_Factor_QP'")
    print(f"✓ Factor Correlation with Target: {super_factor_corr:.4f}")
    recommended_factors = cols_qp

print("\n" + "="*80)



BEST METHOD SELECTION

Detailed Scores:
--------------------------------------------------------------------------------

Baseline MIP (k=5)
  Avg |Correlation|:   0.1118
  Max |Correlation|:   0.3041
  # Features:          5
  Interpretability:    9/10
  Stability:           6/10
  Composite Score:     0.4947

Time-Series CV (k*=1)
  Avg |Correlation|:   0.3041
  Max |Correlation|:   0.3041
  # Features:          1
  Interpretability:    8/10
  Stability:           9/10
  Composite Score:     0.6317

Penalized MIP (λ=0.05)
  Avg |Correlation|:   0.3041
  Max |Correlation|:   0.3041
  # Features:          1
  Interpretability:    7/10
  Stability:           7/10
  Composite Score:     0.5417

Super-Factor (QP)
  Avg |Correlation|:   0.2533
  Max |Correlation|:   0.2533
  # Features:          2
  Interpretability:    5/10
  Stability:           8/10
  Composite Score:     0.4913

RANKINGS BY CRITERION

1. Predictive Power (Avg |Correlation|):
   1. Time-Series CV (k*=1)                